### Demo 1: 

- Sistema de preguntas y respuestas con un pipeline simple con un lector y un buscador

In [3]:
from haystack.nodes import FARMReader, BM25Retriever
from haystack.pipelines import ExtractiveQAPipeline
from haystack.document_stores import InMemoryDocumentStore

# Paso 1: Configurar el DocumentStore con BM25 habilitado
document_store = InMemoryDocumentStore(use_bm25=True)

# Paso 2: Agregar documentos al DocumentStore
documents = [
    {"content": "Lima es la capital de Perú.", "meta": {"source": "Wikipedia"}},
    {"content": "El Machu Picchu es una de las maravillas del mundo.", "meta": {"source": "Wikipedia"}},
    {"content": "El Amazonas es el río más caudaloso del mundo.", "meta": {"source": "Wikipedia"}},
]
document_store.write_documents(documents)

# Paso 3: Configurar un Retriever (BM25 en este caso)
retriever = BM25Retriever(document_store=document_store)

# Paso 4: Configurar un Reader (modelo preentrenado)
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2")

# Paso 5: Crear un pipeline de preguntas y respuestas
pipeline = ExtractiveQAPipeline(reader=reader, retriever=retriever)

# Paso 6: Realizar una consulta
query = "¿Cuál es la capital de Perú?"
result = pipeline.run(query=query, params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 3}})

# Paso 7: Imprimir resultados
print("Respuestas:")
for answer in result["answers"]:
    print(f"Respuesta: {answer.answer}, Puntuación: {answer.score}")

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  2.40 Batches/s]

Respuestas:
Respuesta: Lima, Puntuación: 0.9510083198547363
Respuesta: El Machu Picchu, Puntuación: 0.19514483213424683
Respuesta: El Amazonas, Puntuación: 0.08661656826734543


### Demo 2:
- Busqueda semántica en documentos

In [7]:
from haystack.nodes import DensePassageRetriever, FARMReader
from haystack.pipelines import ExtractiveQAPipeline
from haystack.document_stores import FAISSDocumentStore

# Paso 1: Configurar el DocumentStore
document_store = FAISSDocumentStore(faiss_index_factory_str="Flat")

# Paso 2: Agregar documentos al DocumentStore
documents = [
    {"content": "La inteligencia artificial está transformando la industria tecnológica.", "meta": {"source": "Artículo 1"}},
    {"content": "Los modelos de lenguaje como ChatGPT pueden responder preguntas y generar texto.", "meta": {"source": "Artículo 2"}},
    {"content": "La computación cuántica es una tecnología emergente con gran potencial.", "meta": {"source": "Artículo 3"}},
]
document_store.write_documents(documents)

# Paso 3: Configurar el Retriever denso
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
    use_gpu=True
)

# Paso 4: Actualizar representaciones de los documentos
document_store.update_embeddings(retriever)

# Paso 5: Configurar un Reader
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2")

# Paso 6: Crear el pipeline
pipeline = ExtractiveQAPipeline(reader=reader, retriever=retriever)

# Paso 7: Realizar una consulta
query = "¿Qué está haciendo la inteligencia artificial en la industria?"
result = pipeline.run(query=query, params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 3}})

# Paso 8: Imprimir resultados
print("Respuestas:")
for answer in result["answers"]:
    print(f"Respuesta: {answer.answer}, Puntuación: {answer.score}")


Writing Documents: 10000it [00:00, 1263496.81it/s]      
Documents Processed: 10000 docs [00:00, 260365.38 docs/s]    
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 35.41 Batches/s]

Respuestas:
Respuesta: tecnológica, Puntuación: 0.3722141683101654
Respuesta: gran potencial, Puntuación: 0.042410317808389664
Respuesta: ChatGPT pueden responder preguntas y generar texto., Puntuación: 0.0036275871098041534
